# Files

In [ ]:
# nro de archivo a crear
nro_de_archivo = 3

# existente
Path_linkedin_empresas = 'linkedin_empresas_mkt_11a50empleados.csv'

In [ ]:
df = pd.read_csv(Path_linkedin_empresas)
df = df['Column_Name']
len(df)

In [ ]:
# buscar un divisor redondo para el df
div_df = int(len(df)/5)
div_df

In [ ]:
df_1 = df[0:div_df]
df_2 = df[div_df:div_df*2]
df_3 = df[div_df*2:div_df*3]
df_4 = df[div_df*3:div_df*4]
df_5 = df[div_df*4:div_df*5]

dataframes = [df_1, df_2, df_3, df_4, df_5]

print(len(df_1)+len(df_2)+len(df_3)+len(df_4)+len(df_5))

# Buscar en google a las personas de cada empresa

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from lxml.html import fromstring
import urllib.request
import requests
import numpy as np
import time
np.random.seed(15)

In [ ]:
def GetFirstLinkedinPersonProfile (soup):
    aux = ''
    for link in soup.find_all('a'):
        if str(link).find("linkedin.com/in/") != -1 and str(link).find("https://") != -1 and str(link).find("google") == -1:
            aux = link.get('href')[link.get('href').find('https://'):link.get('href').find('&sa')]
            print(aux)
            return aux

In [ ]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return cycle(proxies)

In [ ]:
import requests
from itertools import cycle
import traceback
def GetPoolProxisFromProxisCsvOrDoGet_proxies ():
    try:
        proxies_df = pd.read_csv('proxies.csv', names = ["proxy"])
        proxy_pool = cycle(proxies_df["proxy"])
        return proxy_pool
    except:
        proxies = get_proxies()
        proxy_pool = cycle(proxies)
        return proxy_pool

In [ ]:
def nro_random(multiplicador, piso = 0.01):
    tiempo = 0
    while tiempo < piso:
        tiempo = np.random.random() * multiplicador
    return tiempo 

In [ ]:
%%time

df_aux = dataframes[nro_de_archivo-1]
data_1 = []

a = 'https://www.google.com.ar/search?source=hp&ei=7oxuX7uUL_m95OUPmOWfiAg&q=site%3Alinkedin.com%2Fin%2F+AND+%28founder+OR+CEO+OR+fundador%29+AND+%22'
b = '%22&oq=site%3Alinkedin.com%2Fin%2F+AND+%28founder+OR+CEO+OR+fundador%29+AND+%22bakian-growthmarketing%22&gs_lcp=CgZwc3ktYWIQA1DzD1iPE2DkFmgAcAB4AoABAIgBAJIBAJgBBaABAaoBB2d3cy13aXo&sclient=psy-ab&ved=0ahUKEwi7w9nvyYXsAhX5HrkGHZjyB4EQ4dUDCAc&uact=5'
indice = len('https://www.linkedin.com/company/')


#proxy_pool = GetPoolProxisFromProxisCsvOrDoGet_proxies()

#i = 0
#proxy_ok = True
#proxy = next(proxy_pool)

for item in df_aux:
    #
    '''if proxy_ok != True:
        proxy = next(proxy_pool)   
        
    print("Request #%d"%i)'''
    #
    #item = df_aux[i]
    url = a + item[indice:] + b
    
    '''try:
        proxyDict = { 
              "http"  : "http://" + proxy, 
              "https" : "https://" + proxy, 
              "ftp"   : "ftp://" + proxy
            }'''
    #html = requests.get(url, proxies = proxyDict)
    html = requests.get(url)
    '''except:
        proxy_ok = False
        continue'''
    html_doc = str(html.content)
    
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    link = GetFirstLinkedinPersonProfile(soup)
    
    #if link != "None":
    data_1.append(link)

    time.sleep(nro_random(13, 3))
    '''else:
        proxy_ok = False'''
    

In [ ]:
dataframe = pd.DataFrame(data_1, columns = ["link_contacto"])
dataframe["link_empresa"] = dataframes[nro_de_archivo-1].reset_index(drop=True)
dataframe.head(3)

In [ ]:
dataframe.to_csv("data_N_ok/data_"+str(nro_de_archivo) +".csv")